In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
hv.extension('bokeh')

In [ ]:
data_file = '../data/iris.data'
colnames = [
    'sepal length',
    'sepal width',
    'petal length',
    'petal width',
    'species'
]
iris = pd.read_csv(data_file, names=colnames, index_col=False)
iris.head()

# Annotate data

* Type - data type (int, float, string, or custom class)
* Dimensions - these are dimensions in the context of the type of plot
  * kdims - key dimensions (analagous to independent variable in math)
  * vdims - value dimensions (analagous to dependent variable in math)
* Group/Label - These annotations provide human-readable descriptions of elements that are used to label plots and select related sets of elements

In [ ]:
# this is a very explicit way of annotating dimensions
# among other things, type annotations occur here
# the label annotation here is for the specific data column
sepal_length = hv.Dimension('sepal length', label='Sepal length', type=float, unit='cm')
sepal_width = hv.Dimension('sepal width', label='Sepal width', type=float, unit='cm')
species = hv.Dimension('species', label='Iris species', type=str, values=iris['species'].unique().tolist())

# key dimensions and value dimensions exist in the context of a plot
# different plot types require different numbers of kdims and vdims
# Group and Label annotations provided here bring context to each plot
# when multiple plots are composed together
sepals = hv.Scatter(
    iris,
    kdims=[sepal_length],
    vdims=[sepal_width, species],
    group='Iris',
    label='Sepal dimensions')

sepals.opts(color='species', cmap='Category10', width=400, height=400)

sepals

*Warning: HoloViews is a big project integrating several other rapidly evolving libraries. It can be tricky to get things working well. Here adding the `type` annotations activates a code path which is currently buggy. It's also not strictly necessary.*

Your experience may vary depending on platform and whether you're using `conda` or `pip`.

## Help system

HoloViews provides the `hv.help()` function which can give some guidance for how to use different features. It can also be used to examine existing plot elements.

In [ ]:
hv.help(hv.Dimension)

In [ ]:
hv.help(sepal_length)

# Composition

In [ ]:
sepl_sepw = hv.Scatter(
    iris,
    kdims=('sepal length', 'Iris Sepal Length'),
    vdims=[
        ('sepal width', 'Width in cm'),
        'species'
    ],
    group='Sepal Length',
    label='Sepal Width')
sepl_petw = hv.Scatter(
    iris,
    kdims=('sepal length', 'Iris Sepal Length'),
    vdims = [
        ('petal width', 'Width in cm'),
        'species'
    ],
    group='Sepal Length',
    label='Petal Width')

sepl_sepw.opts(color='species', cmap='Category10', size=6, width=400, height=400)
sepl_petw.opts(color='species', cmap='Category10', size=6, width=400, height=400, marker='square')

# the '+' operator can combine multiple plots without restriction
sepl_sepw + sepl_petw

In [ ]:
# the '*' operator will overlay plots and so the plots must have a common set of kdims
sepl_sepw * sepl_petw

# Layout

The `+` operator generates a `Layout`. When multiple plot elements in a layout share kdims, the axis on which they are plotted are linked for pan and zoom operations.

In [ ]:
# generate a list from 0 to 9.9 in steps of 0.1
xs = [0.1* i for i in range(100)]

# list of curve plots and a list of scatter plots; 2 of each
# note here the dimension names are left as the default, x and y
curve_list = [
    hv.Curve((xs,
              [np.sin(f*x) for x in xs]))
    for f
    in [0.5, 0.75]]
scatter_list = [
    hv.Scatter((xs[::5], f*np.linspace(0, 1, 20))) 
    for f 
    in [-0.5, 0.5]
]

# the addition of .cols(2) wraps the display at 2 columns
layout = hv.Layout(curve_list + scatter_list).cols(2)
layout

In [ ]:
print(layout)

In [ ]:
# components of the layout are individually addressable
layout.Curve.I

# Overlay

The `*` operator creates an overlay. In order for this to work, the individual components must have kdims in common. 

In [ ]:
overlay = hv.Overlay(curve_list + scatter_list)
overlay

**Note:** The `+` operator here and in the `hv.Layout()` example above are operating on lists not hv objects. The code below uses the `*` to create an overlay from `hv` components and is compatible to calling `hv.Overlay()` with a list of components.

In [ ]:
overlay = curve_list[0] * curve_list[1] * scatter_list[0] * scatter_list[1]
overlay

# Customizing with Options



In [ ]:
sepals = hv.Scatter(
    iris,
    kdims='sepal length',
    vdims=[
        'sepal width',
        'species'
    ])
petals = hv.Scatter(
    iris,
    kdims='petal length',
    vdims = [
        'petal width',
        'species'
    ])

# Inline options
sepals.opts(color='species', cmap='Category10', size=6, width=400, height=400)
petals.opts(color='species', cmap='Category10', size=6, width=400, height=400)

sepals + petals

In [ ]:
# Here are plots with just the bare minimum specifying which data to use
sepals = hv.Scatter(
    iris,
    kdims='sepal length',
    vdims=[
        'sepal width',
        'species'
    ])
petals = hv.Scatter(
    iris,
    kdims='petal length',
    vdims = [
        'petal width',
        'species'
    ])

# Create a layout with our two plots
layout = sepals + petals
layout

In [ ]:
# options can be specified completely separately as an independent object
scatteropts = hv.opts.Scatter(color='species', cmap='Category10', size=6, width=400, height=400)

# then applied to layouts or plots
layout.opts(scatteropts)
layout

# Recap

Holoviews is set up to create composable elements. This lets the user *separate concerns* in their code. The details of data can cleanly be separated from the details of a layout or styling. Conversely, not all components are required to be enumerated in full detail -- the library will get out of your way when you don't need all the control.

* Dimensions with annotations
  - as a tuple: ('dataname', 'Extended label')
  - as a `hv.Dimension()`
* Plot elements
  - plot type
  - data
  - kdims
  - vdims
* Layouts and Overlays
* Options

There are other powerful components but